In [29]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key
from config import g_key

In [30]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Mandera,KE,3.94,41.87,78.08,67,36,19.69,scattered clouds,0.0,0.00
1,1,Nome,US,64.50,-165.41,68.00,63,1,12.75,clear sky,0.0,0.00
2,2,Vaini,TO,-21.20,-175.20,73.40,73,75,17.22,light intensity drizzle,0.0,0.00
3,3,Punta Arenas,CL,-53.15,-70.92,33.80,84,72,8.05,light snow,0.0,0.13
4,4,Maceio,BR,-9.67,-35.74,73.40,88,75,1.12,broken clouds,0.0,0.00


In [31]:
# Configure gmaps to use API key.
gmaps.configure(api_key=g_key)

In [32]:
city_data_df.dtypes

Unnamed: 0               int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [33]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Raining?
rain_q = (input("Do you want it to be raining? (yes/no) "))
# Snowing?
snow_q = (input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 89
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [34]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Rain (inches)"] == 0 if rain_q == "no" else city_data_df["Rain (inches)"] > 0) &
                                       (city_data_df["Snow (inches)"] == 0 if snow_q == "no" else city_data_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Mandera,KE,3.94,41.87,78.08,67,36,19.69,scattered clouds,0.0,0.0
24,24,Albany,US,42.60,-73.97,82.99,53,10,5.44,clear sky,0.0,0.0
28,28,Salalah,OM,17.02,54.09,80.60,94,90,3.36,drizzle,0.0,0.0
30,30,Victoria,HK,22.29,114.16,88.00,74,11,4.00,few clouds,0.0,0.0
36,36,Rongcheng,CN,30.63,117.85,84.00,99,99,3.18,overcast clouds,0.0,0.0
44,44,Tual,ID,-5.67,132.75,79.07,81,44,22.41,scattered clouds,0.0,0.0
45,45,Nizhniy Kuranakh,RU,58.84,125.49,85.64,44,28,1.92,scattered clouds,0.0,0.0
52,52,Faanui,PF,-16.48,-151.75,79.02,75,100,14.72,overcast clouds,0.0,0.0
53,53,Coos Bay,US,43.37,-124.22,75.99,68,1,21.92,clear sky,0.0,0.0
68,68,Shingu,JP,33.73,135.98,75.20,69,75,1.12,shower rain,0.0,0.0


In [35]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, description and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mandera,KE,78.08,scattered clouds,3.94,41.87,
24,Albany,US,82.99,clear sky,42.60,-73.97,
28,Salalah,OM,80.60,drizzle,17.02,54.09,
30,Victoria,HK,88.00,few clouds,22.29,114.16,
36,Rongcheng,CN,84.00,overcast clouds,30.63,117.85,
44,Tual,ID,79.07,scattered clouds,-5.67,132.75,
45,Nizhniy Kuranakh,RU,85.64,scattered clouds,58.84,125.49,
52,Faanui,PF,79.02,overcast clouds,-16.48,-151.75,
53,Coos Bay,US,75.99,clear sky,43.37,-124.22,
68,Shingu,JP,75.20,shower rain,33.73,135.98,


In [36]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [37]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mandera,KE,78.08,scattered clouds,3.94,41.87,Kornesh Hotel and Lodging
24,Albany,US,82.99,clear sky,42.60,-73.97,
28,Salalah,OM,80.60,drizzle,17.02,54.09,HAMDAN PLAZA HOTEL
30,Victoria,HK,88.00,few clouds,22.29,114.16,Mini Hotel Central
36,Rongcheng,CN,84.00,overcast clouds,30.63,117.85,Shell Shandong Liaocheng Development District ...
44,Tual,ID,79.07,scattered clouds,-5.67,132.75,Grand Vilia Hotel
45,Nizhniy Kuranakh,RU,85.64,scattered clouds,58.84,125.49,Park Pobedy
52,Faanui,PF,79.02,overcast clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
53,Coos Bay,US,75.99,clear sky,43.37,-124.22,Best Western Holiday Hotel
68,Shingu,JP,75.20,shower rain,33.73,135.98,Shingu UI Hotel


In [39]:
# Get the maximum temperature.
max_temp = hotel_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [43]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

In [44]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [45]:
# Add a marker layer for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [46]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="")